In [1]:
from cobra import design
import cobra

In [2]:
ijo = cobra.io.load_json_model('/home/sbrg-cjlloyd/Desktop/ecoli_M_models/iJO1366.json')

In [37]:
from collections import defaultdict
def dual_model(cons_model, dual_max = 1000., cons=True):
    model = cons_model.copy()
    if not cons:
        mult = -1
    else:
        mult = 1
    dual_model = cobra.Model('%s_dual' % model.id)
    reaction_dict = defaultdict(dict)
    for reaction in model.reactions:
        dual_constraint = cobra.Metabolite(reaction.id + '_constraint')
        
        for met, coeff in reaction.metabolites.items():
            reaction_dict[met.id][dual_constraint.id] = coeff
        # to change from >= to <=, multiply both sides of (x) by -1
        dual_constraint._constraint_sense = 'L'
        dual_constraint._bound = mult * reaction.objective_coefficient
        dual_model.add_metabolites(dual_constraint)
        
    for met in model.metabolites:
        dual_variable = cobra.Reaction(met.id + '_variable')
        
        dual_model.add_reaction(dual_variable)
        # to change from >= to <=, multiply both sides of (x) by -1
        for rxn, coeff in reaction_dict[met.id].items():
            dual_variable.add_metabolites({rxn: -coeff})
        
        dual_variable.upper_bound = dual_max
        dual_variable.lower_bound = 0.
        
        dual_variable.objective_coefficient = -mult * met._bound
    
    return dual_model

In [3]:
ijo_dual = design.dual_problem(ijo)
ijo_dual.optimize('minimize')

<Solution 0.98 at 0x7f6a36ff1fd0>

In [4]:
ijo_dual_2 = design.dual_problem(ijo_dual, even_dual=True, already_canonical=True)
ijo_dual_2.optimize()

TypeError: dual_problem() got an unexpected keyword argument 'even_dual'

In [28]:
ijo_con = cobra.manipulation.canonical_form(ijo)

In [29]:
ijo_dual = dual_model(ijo_con, cons=False)

In [30]:
ijo_dual.optimize('minimize')

<Solution 0.98 at 0x7f98e69b7190>

In [31]:
ijo_dual_dual = dual_model(ijo_dual)

In [32]:
ijo_dual_dual.optimize(solver='cplex')

<Solution 0.98 at 0x7f98e0751090>

In [24]:
ijo_dual_dual.metabolites.EX_glc__D_e_reverse__UB_constraint_variable_constraint._bound

10.0

In [38]:
ijo_dual_dual_dual = dual_model(ijo_dual_dual)

In [40]:
ijo_dual_dual_dual.optimize('minimize', solver='cplex')

<Solution 0.98 at 0x7f98c21ae2d0>

In [26]:
ijo_dual_dual.reactions.EX_glc__D_e_reverse_constraint_variable.reaction

'EX_glc__D_e_reverse__UB_constraint_variable_constraint + glc__D_e_variable_constraint --> EX_glc__D_e_reverse__LB_constraint_variable_constraint + glc__D_e__GE_constraint_variable_constraint'

In [108]:
for r in ijo_dual.metabolites.BIOMASS_Ec_iJO1366_core_53p95M_constraint.reactions:
    print r.id, r._metabolites[ijo_dual.metabolites.BIOMASS_Ec_iJO1366_core_53p95M_constraint]

nh4_c_variable 0.013013
leu__L_c__GE_constraint_variable -0.450531
pydx5p_c_variable 0.000223
h_c_variable -53.95
pe161_c__GE_constraint_variable -0.054154
cu2_c_variable 0.000709
fad_c__GE_constraint_variable -0.000223
ca2_c__GE_constraint_variable -0.005205
his__L_c__GE_constraint_variable -0.094738
4fe4s_c_variable 0.00026
cobalt2_c_variable 2.5e-05
lys__L_c_variable 0.343161
thmpp_c_variable 0.000223
fe3_c__GE_constraint_variable -0.007808
pheme_c_variable 0.000223
pe161_c_variable 0.054154
fad_c_variable 0.000223
fe2_c__GE_constraint_variable -0.006715
fe2_c_variable 0.006715
cl_c__GE_constraint_variable -0.005205
kdo2lipid4_e__GE_constraint_variable -0.019456
ni2_c_variable 0.000323
arg__L_c__GE_constraint_variable -0.295792
asn__L_c__GE_constraint_variable -0.241055
phe__L_c__GE_constraint_variable -0.185265
fe3_c_variable 0.007808
ribflv_c__GE_constraint_variable -0.000223
dttp_c_variable 0.026166
ribflv_c_variable 0.000223
cl_c_variable 0.005205
ile__L_c_variable 0.290529
val_

In [109]:
ijo_dual_dual.objective

{<Reaction prbatp_c__GE_constraint_variable__UB_constraint_variable at 0x7f9e41480090>: 1000.0,
 <Reaction ENTERES2__LB_constraint_variable__UB_constraint_variable at 0x7f9e40a2f2d0>: 1000.0,
 <Reaction preq0_c__GE_constraint_variable__UB_constraint_variable at 0x7f9e41480110>: 1000.0,
 <Reaction ECOAH7__LB_constraint_variable__UB_constraint_variable at 0x7f9e40a15590>: 1000.0,
 <Reaction preq1_c__GE_constraint_variable__UB_constraint_variable at 0x7f9e41480190>: 1000.0,
 <Reaction PGL__LB_constraint_variable__UB_constraint_variable at 0x7f9e40340050>: 1000.0,
 <Reaction prfp_c__GE_constraint_variable__UB_constraint_variable at 0x7f9e41480210>: 1000.0,
 <Reaction mnl_e_variable__UB_constraint_variable at 0x7f9e4196ab10>: 1000.0,
 <Reaction prlp_c__GE_constraint_variable__UB_constraint_variable at 0x7f9e41480290>: 1000.0,
 <Reaction pro__L_c__GE_constraint_variable__UB_constraint_variable at 0x7f9e41480310>: 1000.0,
 <Reaction progly_c__GE_constraint_variable__UB_constraint_variable at 

In [8]:
2%2

0

In [14]:
if 0:
    print 'a'